## NN Linear Regression

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error

#### Google colab

* How to check if using Google Colab to upload data files

In [ ]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import files
    uploaded = files.upload()

In [ ]:
howell = pd.read_csv("Howell.csv",sep=';')
adult = howell.query("age > 17")
adult.tail()

In [ ]:
X = adult.weight.values.reshape(-1,1)
#X = adult.loc[:,['weight','age']]
y = adult.height.values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=1234)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = scaler.fit_transform(y_train)
y_test = scaler.transform(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## PyTorch

In [ ]:
import torch
import torch.nn as nn
torch.__version__

In [ ]:
torch.manual_seed(1)
# Create the linear regression model
#model = nn.Linear(1, 1)

class LR(nn.Module):
    def __init__(self,num_in,num_out):
        super(LR, self).__init__() 
        self.linear = torch.nn.Linear(num_in, num_out)  
  
    def forward(self, x): 
        
        y_pred = self.linear(x) 
        return y_pred 
    
model = LR(X_train.shape[1],1)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

In [ ]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [ ]:
# PyTorch uses float32 by default
# Numpy creates float64 by default
feats = torch.from_numpy(X_train.astype(np.float32)).to(device)
target = torch.from_numpy(y_train.astype(np.float32)).to(device)
test_input = torch.from_numpy(X_test.astype(np.float32)).to(device)
test_target = torch.from_numpy(y_test.astype(np.float32)).to(device)

feats.shape,target.shape,test_input.shape,test_target.shape

In [ ]:
# Train the model
n_epochs = 30

train_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)

for i in range(n_epochs):

    optimizer.zero_grad()  # zero the parameter gradients
    outputs = model(feats) # Forward pass
    loss = criterion(outputs, target) # Calculate loss
    loss.backward()        # Backward pass
    optimizer.step()       # Update weights
    train_losses[i] = loss.item() # Log loss
    
    outputs_test = model(test_input) # Get test loss
    loss_test = criterion(outputs_test, test_target)
    test_losses[i] = loss_test.item()
    
    print(f'Epoch {i+1}/{n_epochs}, Train Loss: {loss.item():.4f} Test Loss: {loss_test.item():.4f}')

In [ ]:
# Plot the loss per iteration
plt.plot(train_losses, label='train loss')
plt.plot(test_losses, label='test loss')
plt.legend();

In [ ]:
foo = model(torch.from_numpy(X_test.astype(np.float32)).to(device))
type(foo)

In [ ]:
predicted = foo.cpu().detach().numpy()
predicted.shape

In [ ]:
predicted2 = outputs_test..cpu().detach().numpy()
predicted2.shape

In [ ]:
if X_test.shape[1] == 1:
    plt.scatter(X_test, y_test, label='Original data')
    plt.plot(X_test, predicted, 'r', label='Fitted line')
    plt.plot(X_test, predicted2, label='Fitted2 line')

    plt.legend();

In [ ]:
r2_score(y_test,predicted),np.sqrt(mean_squared_error(y_test,predicted))

In [ ]:
r2_score(y_test,predicted2),np.sqrt(mean_squared_error(y_test,predicted2))

In [ ]:
list(model.parameters())

### Two layers

In [ ]:
X = adult.loc[:,['weight','age']].values
y = adult.height.values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=1234)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = scaler.fit_transform(y_train)
y_test = scaler.transform(y_test)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
torch.manual_seed(1)

class LR2(nn.Module):
    def __init__(self,num_in,num_out):
        super(LR2, self).__init__() 
        self.hid1 = nn.Linear(num_in,8)
        self.outp = nn.Linear(8,num_out)
        
  
    def forward(self, x): 
        z = nn.ReLU()(self.hid1(x))
        y_pred = self.outp(z)
        return y_pred 
    
model = LR2(X_train.shape[1],1)
model.to(device)

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [ ]:
feats = torch.from_numpy(X_train.astype(np.float32)).to(device)
target = torch.from_numpy(y_train.astype(np.float32)).to(device)
test_input = torch.from_numpy(X_test.astype(np.float32)).to(device)
test_target = torch.from_numpy(y_test.astype(np.float32)).to(device)
feats.shape,target.shape,test_input.shape,test_target.shape

In [ ]:
# Train the model
n_epochs = 30

train_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)

for i in range(n_epochs):

    optimizer.zero_grad()  # zero the parameter gradients
    outputs = model(feats) # Forward pass
    loss = criterion(outputs, target) # Calculate loss
    loss.backward()        # Backward pass
    optimizer.step()       # Update weights
    train_losses[i] = loss.item() # Log loss
    
    outputs_test = model(test_input) # Get test loss
    loss_test = criterion(outputs_test, test_target)
    test_losses[i] = loss_test.item()
    
    print(f'Epoch {i+1}/{n_epochs}, Train Loss: {loss.item():.4f} Test Loss: {loss_test.item():.4f}')

In [ ]:
# Plot the loss per iteration
plt.plot(train_losses, label='train loss')
plt.plot(test_losses, label='test loss')
plt.legend();

In [ ]:
foo = model(torch.from_numpy(X_test.astype(np.float32))).to(device)
predicted = foo.cpu().detach().numpy()
predicted.shape

In [ ]:
r2_score(y_test,predicted),np.sqrt(mean_squared_error(y_test,predicted))

In [ ]:
list(model.parameters())

## Tensorflow Linear Regression

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
tf.__version__

In [ ]:
model = Sequential()

model.add(Dense(units=8,activation='relu'))
model.add(Dense(units=1))

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model.fit(x=X_train, 
          y=y_train, 
          epochs=30,
          validation_data=(X_test, y_test), verbose=1
          )

In [ ]:
model_loss = pd.DataFrame(model.history.history)
model_loss.plot()

In [ ]:
predicted = model.predict_classes(X_test)

In [ ]:
r2_score(y_test,predicted),np.sqrt(mean_squared_error(y_test,predicted))